# SDFusion: Text-guided Generation (txt2shape)

### TODO: add sample results or teaser images

In [1]:
# first set up which gpu to use
import os
gpu_ids = 0
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_ids}"

In [2]:
# import libraries
import numpy as np
from IPython.display import Image as ipy_image
from IPython.display import display
from termcolor import colored, cprint

import torch
print(torch.cuda.is_available())
import torch.backends.cudnn as cudnn
cudnn.benchmark = True
import torchvision.utils as vutils

from models.base_model import create_model
from utils.util_3d import render_sdf, render_mesh, sdf_to_mesh, save_mesh_as_gif

%load_ext autoreload
%autoreload 2

/root/miniconda3/envs/sdfusion/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True


In [3]:
# options for the model. please check `utils/demo_util.py` for more details
from utils.demo_util import SDFusionText2ShapeOpt

seed = 2023
opt = SDFusionText2ShapeOpt(gpu_ids=gpu_ids, seed=seed)
device = opt.device

[*] SDFusionText2ShapeOption initialized.


In [4]:
# enable proxy for huggingface

import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [5]:
# initialize SDFusion model
ckpt_path = '/root/autodl-tmp/SDFusion/logs_home/continue-2024-04-05T17-15-19-sdfusion-txt2shape-text2shape-all-LR1e-5-clean-code/ckpt/df_steps-latest.pth'
vqvae_path = '/root/autodl-tmp/SDFusion/logs_home/continue-2024-03-27T11-26-32-vqvae-snet-all-res64-LR1e-4-T0.2-release/ckpt/vqvae_epoch-best.pth'
vqvae_path = '/root/autodl-tmp/SDFusion/saved_ckpt/vqvae-snet-all.pth'
opt.init_model_args(ckpt_path=ckpt_path, vq_ckpt_path=vqvae_path)

SDFusion = create_model(opt)
cprint(f'[*] "{SDFusion.name()}" loaded.', 'cyan')

Working with z of shape (1, 3, 16, 16, 16) = 12288 dimensions.
[*] VQVAE: weight successfully load from: /root/autodl-tmp/SDFusion/saved_ckpt/vqvae-snet-all.pth
[*] weight successfully load from: /root/autodl-tmp/SDFusion/logs_home/continue-2024-04-05T17-15-19-sdfusion-txt2shape-text2shape-all-LR1e-5-clean-code/ckpt/df_steps-latest.pth
[*] setting ddim_steps=100
[*] Model has been created: SDFusion-Text2Shape-Model
[*] "SDFusion-Text2Shape-Model" loaded.


In [6]:
import torch
import pytorch3d
from pytorch3d.ops import sample_points_from_meshes

def mesh_to_sdf(mesh, resolution=64, bounds=None, batch_size=1024):
    device = mesh.device
    # If bounds are not specified, compute them from the mesh
    if bounds is None:
        bounds = torch.stack([mesh.verts_packed().min(0)[0], mesh.verts_packed().max(0)[0]]).T
        padding = (bounds[:, 1] - bounds[:, 0]) * 0.1
        bounds[:, 0] -= padding
        bounds[:, 1] += padding
    
    # Create a grid of points where the SDF will be evaluated
    x = torch.linspace(bounds[0, 0], bounds[0, 1], resolution, device=device)
    y = torch.linspace(bounds[1, 0], bounds[1, 1], resolution, device=device)
    z = torch.linspace(bounds[2, 0], bounds[2, 1], resolution, device=device)
    grid = torch.stack(torch.meshgrid(x, y, z), dim=-1).reshape(-1, 3)
    
    # Sample points from the mesh surface
    num_samples = resolution ** 3  # Adjust the number of samples as needed
    surface_points = sample_points_from_meshes(mesh, num_samples).squeeze(0)

    # Initialize SDF tensor
    sdf = torch.full((grid.size(0),), float('inf'), device=device)
    
    # Process each batch of grid points
    for start in range(0, grid.size(0), batch_size):
        end = min(start + batch_size, grid.size(0))
        distances = torch.cdist(grid[start:end], surface_points)
        min_distances, _ = torch.min(distances, dim=1)
        sdf[start:end] = min_distances
    
    # Placeholder for inside/outside determination
    sign = torch.ones_like(sdf)  # All outside for simplicity

    # Convert distances to SDF values
    sdf *= sign

    # Reshape to the resolution grid
    sdf = sdf.view(resolution, resolution, resolution)

    return sdf

## SDFusion: text-guided generation (txt2shape)

In [23]:
import pytorch3d
from tqdm import tqdm

# mesh_dir = '/root/autodl-tmp/SDFusion/test_results_nopre_small_dset/obj/'
# mesh_dir = '/root/autodl-tmp/SDFusion/data/ShapeNet/ShapeNetCore.v1/01'
# mesh_path = os.listdir(mesh_dir)
# mesh_path = [os.path.join(mesh_dir, m_p) for m_p in mesh_path if m_p.endswith('obj')]
# mesh_path = [os.path.join(mesh_dir, m_p, 'model.obj') for m_p in mesh_path if m_p.startswith('b')]
# meshes = pytorch3d.io.load_objs_as_meshes(mesh_path, device=device)

# print("meshes loaded")

# sdfs = []
# for mesh in tqdm(meshes):
#     sdf = mesh_to_sdf(mesh, resolution=64)
#     sdfs.append(sdf)

# sdfs = torch.stack(sdfs).unsqueeze(1).unsqueeze(1)
# print(sdfs.shape)

import h5py
sdf_dir = "/root/autodl-tmp/SDFusion/data/results_pretrained_small_dset/SDF_v1/resolution_64/01/"
sdf_paths = os.listdir(sdf_dir)

sdfs = []
for sub_dir in sdf_paths:
    if not sub_dir.startswith('b'):
        continue
    sdf_path = os.path.join(sdf_dir, sub_dir, "ori_sample_grid.h5")
    h5_f = h5py.File(sdf_path, 'r')
    sdf = h5_f['pc_sdf_sample'][:].astype(np.float32)
    sdf = torch.Tensor(sdf).view(1, 64, 64, 64).to(device)
    sdf = sdf.unsqueeze(0).unsqueeze(0)
    sdfs.append(sdf)

sdfs = torch.cat(sdfs, dim=0)
print(sdfs.shape)

print("sdfs loaded")

codes = []
SDFusion.eval()
SDFusion.vqvae.eval()
with torch.no_grad():
    # SDFusion.vqvae.encode(sdf)
    for sdf in tqdm(sdfs):
        code = SDFusion.vqvae.encode_no_quant(sdf)
        codes.append(code)

# calculate mean and std for FID, but here we save first for later FID calculation
codes = torch.cat(codes, dim=0)
torch.save(codes, '/root/autodl-tmp/SDFusion/data/results_pretrained_small_dset/codes.pth')

torch.Size([101, 1, 1, 64, 64, 64])
sdfs loaded


100%|██████████| 101/101 [00:01<00:00, 70.08it/s]
